# Ishiki Detector
君の意識は高いか。
文章の意識高さを判定するニューラルネットワークモデル。

## Setup
- jupyter でこの ipynb を立ち上げてください
- 適当な場所に以下の2つのテキストファイルを置いてください（改行区切り）
    - 意識高い文を20万個くらい集めたテキストファイル
    - 意識低い文を20万個くらい集めたテキストファイル
- 私は前者は [TED](http://logmi.jp/tag/ted) の文章、後者は Twitter の文章を使いました。

## Run
### Training
上から順に Train まで実行してください。

- pos_path, neg_path は Setup で集めたデータのパスに書き換えてください。
- save_dir はモデルを保存するディレクトリパスに書き換えてください。

### Restore & Predict
Preprocess Data 及び、 Restore 以降を実行してください。

- prediction_text_list は判定をしたい文章を改行区切りで入力してください
- uses_sort を True にすれば結果が意識低い順にソートされます

# Preprocess Data

In [1]:
import os
import numpy as np

In [2]:
pos_path = '/home/harumitsu.nobuta/tmp/ishiki_detector/ishikitakai.txt'
neg_path = '/home/harumitsu.nobuta/tmp/ishiki_detector/ishikihikui.txt'
validation_num = 10000

In [3]:
char2id_dic = {'': 0}  # 文字 -> ID の変換辞書
id2char_dic = {0: ''}  # ID -> 文字の変換辞書

def text2id_list(text):
    return [char2id_dic[c] for c in text]

def id_list2text(id_list):
    return ''.join([id2char_dic[id_] for id_ in id_list])

def update_char_dict(c):
    if c not in char2id_dic:
        new_id = len(char2id_dic)
        char2id_dic[c] = new_id
        id2char_dic[new_id] = c

def load_data(path):
    with open(path) as f:
        raw_text = f.read()
    for c in raw_text:
        update_char_dict(c)
    text_list = [line.strip() for line in raw_text.split()]
    text_list = list(set(text_list))  # 重複を削除
    id_list_list = [text2id_list(text) for text in text_list]
    return id_list_list

def convert_padded_array(id_list_list, max_length):
    return np.array([[0] * (max_length - len(id_list)) + id_list for id_list in id_list_list])

In [4]:
pos_data = load_data(pos_path)
neg_data = load_data(neg_path)
print('pos: {}\nneg: {}'.format(len(pos_data), len(neg_data)))

# pos/neg の数を同じにする
data_num = min(len(pos_data), len(neg_data))
print('result data num: {}'.format(data_num))
pos_data = pos_data[:data_num]
neg_data = neg_data[:data_num]

max_length = max([len(id_list) for id_list in pos_data + neg_data])
print('max_length: {}'.format(max_length))

half_validation_num = int(validation_num / 2)
validation_x = convert_padded_array(pos_data[:half_validation_num] + neg_data[:half_validation_num], max_length)
validation_y = np.array([1] * half_validation_num + [0] * half_validation_num)
train_x = convert_padded_array(pos_data[half_validation_num:] + neg_data[half_validation_num:], max_length)
train_y = np.array([1] * (data_num - half_validation_num) + [0] * (data_num - half_validation_num))

pos: 200000
neg: 221309
result data num: 200000
max_length: 85


# Create Model

In [5]:
embedding_dim = 128
rnn_hidden_dim = 512
dense_layer_num = 2
dense_hidden_dim = 128
dropout = 0.4

In [6]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import GRU
from keras.callbacks import EarlyStopping, TensorBoard

vocab_size = len(char2id_dic)

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(GRU(rnn_hidden_dim, return_sequences=False, dropout=dropout))
for l in range(dense_layer_num):
    model.add(Dense(dense_hidden_dim))
    model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['binary_accuracy'])

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
save_dir = '/home/harumitsu.nobuta/tmp/ishiki_detector/log/base'

# Train

In [ ]:
batch_size = 1000
learning_rate = 0.001
epoch_num = 2
early_stopping_patience = 2

In [ ]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
with open(os.path.join(save_dir, 'model.json'), 'w') as f:
    f.write(model.to_json())

In [ ]:
model.fit(
    train_x,
    train_y,
    batch_size=batch_size,
    epochs=epoch_num,
    validation_data=(validation_x, validation_y),
    callbacks=[
        EarlyStopping(patience=early_stopping_patience),
        TensorBoard(os.path.join(save_dir, 'log')),
    ],
    shuffle=True,
)
model.save_weights(os.path.join(save_dir, 'param.hdf5'))

# Restore

In [8]:
from keras.models import model_from_json
import json
with open(os.path.join(save_dir, 'model.json')) as f:
    model = model_from_json(f.read())
model.load_weights(os.path.join(save_dir, 'param.hdf5'))

# Prediction

In [11]:
prediction_text_list = '''
ミクシィグループは『新しい文化を創る』をミッションに掲げコミュニケーションを軸にした新しい価値の提供により新たな市場の創造に挑戦する企業グループです
データサイエンティストの宿命
世界の平和を保つため、我々は行動しなくてはならないのです
高負荷耐性な設計ができるエンジニアが求められている
我々もやらねばならぬのです
------------------------------
圧倒的当事者意識
学習率のDecayがちょっと遅くなっていると思いますがどのOptimizerを使っていますか
学習率のDecayがちょっと遅いけどOptimizerを使ってるん
------------------------------
ビール飲みたい
業務ほったらかして意識高い判定器作ってるんだけどこれ意識低くない
お腹へってるときに飯テロされると辛い
どうでもええからさっさと学習終われや
誕生日おめでとー
まじありえない
オカメインコとコザクラインコ、どっちがかわいいかまよううううううう
働きたくないでござる
'''.split()
uses_sort = False

pred_id_list_list = [text2id_list(text) for text in prediction_text_list]
pred_x = convert_padded_array(pred_id_list_list, max_length)
result_list = model.predict(pred_x)

pairs = list(zip(prediction_text_list, result_list))
if uses_sort:
    pairs.sort(key=lambda x: x[1][0])
for text, result in pairs:
    pos_rate = result[0]
    prefix = '😎' if pos_rate > 0.5 else '😜'
    print('意識高さ {: 6.1f}% - {}'.format(pos_rate * 100, prefix + text))    

意識高さ  100.0% - 😎ミクシィグループは『新しい文化を創る』をミッションに掲げコミュニケーションを軸にした新しい価値の提供により新たな市場の創造に挑戦する企業グループです
意識高さ   83.4% - 😎データサイエンティストの宿命
意識高さ   99.9% - 😎世界の平和を保つため、我々は行動しなくてはならないのです
意識高さ   94.7% - 😎高負荷耐性な設計ができるエンジニアが求められている
意識高さ   79.2% - 😎我々もやらねばならぬのです
意識高さ  100.0% - 😎------------------------------
意識高さ   57.2% - 😎圧倒的当事者意識
意識高さ   27.1% - 😜学習率のDecayがちょっと遅くなっていると思いますがどのOptimizerを使っていますか
意識高さ    2.1% - 😜学習率のDecayがちょっと遅いけどOptimizerを使ってるん
意識高さ  100.0% - 😎------------------------------
意識高さ    2.6% - 😜ビール飲みたい
意識高さ   10.5% - 😜業務ほったらかして意識高い判定器作ってるんだけどこれ意識低くない
意識高さ    0.3% - 😜お腹へってるときに飯テロされると辛い
意識高さ   43.1% - 😜どうでもええからさっさと学習終われや
意識高さ    1.5% - 😜誕生日おめでとー
意識高さ    5.4% - 😜まじありえない
意識高さ    0.5% - 😜オカメインコとコザクラインコ、どっちがかわいいかまよううううううう
意識高さ   12.9% - 😜働きたくないでござる
